In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import trafficDataUtility as util

In [3]:
# loading data file

In [4]:
path = "C:\\Users\\KangYounKook\\Desktop\\ML_Study\\"

In [5]:
dataset = {}
dataset = util.getDatasetFromFile(path+"sectionInfo.txt")

getDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\sectionInfo.txt


In [6]:
filename = "PeriodTotalAverage.txt"
util.updateDatasetFromFile(path+filename, dataset)

loadDataFromFile  C:\Users\KangYounKook\Desktop\ML_Study\PeriodTotalAverage.txt
update Dataset [move_count,work_count,ratio]


In [7]:
classes = 10

In [13]:
#filename = "input" + str(classes) +"_all.csv"
filename = "input10_all.csv"
data = util.loadCsvInput(path,filename)

loadCsvInput  C:\Users\KangYounKook\Desktop\ML_Study\input10_all.csv


In [14]:
# model

In [17]:
num = 0
selectedList = ['Section697','Section754','Section1212','Section852']
#for id in dataset.keys():
for id in selectedList:

    if id not in data.columns:
        continue
    if dataset[id][5] < 1440:
        continue
    Xa = util.getX(id,data,dataset)
    if Xa is None:
        continue
    Ya = np.array(data[id]).reshape(-1,1)
    num += 1
    print(num, id, '-------------------------')
#    Xn = np.c_[Xa[:1437],Xa[1:1438],Xa[2:1439]]
    Xn = np.c_[Xa[:1437],Xa[1:1438]]
    
#    Xn = Xa[:1437]
#    print(Xn.shape)
    Yn = Ya[3:1440]
#    print(Yn.shape)
    
    Xt = Xn[0:1000]
    Xval = Xn[1000:1437]
    Yt = Yn[0:1000]
    Yval = Yn[1000:1437]
    
    input_size = 10
    X = tf.placeholder(tf.float32, [None, input_size])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W = tf.Variable(tf.random_normal([input_size, classes]), name='weight')
    b = tf.Variable(tf.random_normal([classes]), name='bias')

    logits = tf.matmul(X, W) + b
    hypothesis = tf.nn.softmax(logits)

    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)

    optimizer = tf.train.AdamOptimizer().minimize(cost)
    prediction = tf.argmax(hypothesis, 1)
    correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for step in range(2000):
            sess.run(optimizer, feed_dict={X:Xt, Y:Yt})
#            if step % 2000 == 0:
#                loss, acc = sess.run([cost,accuracy], feed_dict={X:Xt, Y:Yt})
#                print("Step:{:5} Loss:{:.3f} Acc:{:.2%}".format(step, loss, acc))
        print('accuracy:', sess.run(accuracy, feed_dict={X:Xt,Y:Yt}))
        print('accuracy:', sess.run(accuracy, feed_dict={X:Xval,Y:Yval}))
    

1 Section697 -------------------------
accuracy: 0.745
accuracy: 0.775744
2 Section754 -------------------------
accuracy: 0.679
accuracy: 0.675057
3 Section1212 -------------------------
accuracy: 0.551
accuracy: 0.517162
4 Section852 -------------------------
accuracy: 0.719
accuracy: 0.704805


In [269]:
# data preprocessing for input

In [270]:
def model(input_size, classes):
    X = tf.placeholder(tf.float32, [None, input_size])
    Y = tf.placeholder(tf.int32, [None, 1])
    Y_one_hot = tf.one_hot(Y, classes)
    Y_one_hot = tf.reshape(Y_one_hot, [-1, classes])

    W = tf.Variable(tf.random_normal([input_size, classes]), name='weight')
    b = tf.Variable(tf.random_normal([classes]), name='bias')

    logits = tf.matmul(X, W) + b
    hypothesis = tf.nn.softmax(logits)

    cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost_i)
    return hypothesis, cost

In [ ]:
#tf.contrib.lookup.index_table_from_tensor